## Set up

In [ ]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -q fastkaggle

from fastkaggle import *

In [ ]:
comp = 'digit-recognizer'

path = setup_comp(comp, install='fastai fastcore timm nbdev')

In [ ]:
path

In [ ]:
from fastai.vision.all import *
import os
import seaborn as sns
#set_seed(42)

path.ls()

## Looking at the data

In [ ]:
# Load the data
df_train = pd.read_csv(path/"train.csv")
df_test = pd.read_csv(path/"test.csv")

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
Y_train = df_train["label"]

# Drop 'label' column
X_train = df_train.drop(labels = ["label"],axis = 1) 

# free some space
#del train 

Y_train.value_counts().plot(kind='bar')
plt.show()

## Get data into the right format

This part I have adapted from https://www.kaggle.com/code/christianwallenwein/beginners-guide-to-mnist-with-fast-ai

In [ ]:
! mkdir /kaggle/working/train
! mkdir /kaggle/working/test

In [ ]:
TRAIN = Path("/kaggle/working/train")
TEST = Path("/kaggle/working/test")

# Create training directory
for index in range(10):
    try:
        os.makedirs(TRAIN/str(index))
    except:
        pass
# Test whether creating the training directory was successful
sorted(os.listdir(TRAIN))
#Create test directory
try:
    os.makedirs(TEST)
except:
    pass

In [ ]:
def saveDigit(digit, filepath):
    digit = digit.reshape(28,28)
    digit = digit.astype(np.uint8)

    img = Image.fromarray(digit)
    img.save(filepath)
    
# save training images
for index, row in df_train.iterrows():
    
    label,digit = row[0], row[1:]
    
    folder = TRAIN/str(label)
    filename = f"{index}.jpg"
    filepath = folder/filename
    
    digit = digit.values
    
    saveDigit(digit, filepath)
    
# save testing images
for index, digit in df_test.iterrows():

    folder = TEST
    filename = f"{index}.jpg"
    filepath = folder/filename
    
    digit = digit.values
    
    saveDigit(digit, filepath)

## Check images

In [ ]:
# import matplotlib to arrange the images properly
import matplotlib.pyplot as plt

def displayTrainingData():
    fig = plt.figure(figsize=(5,10))
    
    for rowIndex in range(1, 10):
        subdirectory = str(rowIndex)
        path = TRAIN/subdirectory
        images = os.listdir(path)
        for sampleIndex in range(1, 6):
            randomNumber = random.randint(0, len(images)-1)
            image = Image.open(path/images[randomNumber])
            ax = fig.add_subplot(10, 5, 5*rowIndex + sampleIndex)
            ax.axis("off")
            
            plt.imshow(image, cmap='gray')
        
    plt.show()
    
def displayTestingData():
    fig = plt.figure(figsize=(5, 10))
    
    paths = os.listdir(TEST)
    
        
    for i in range(1, 51):
        randomNumber = random.randint(0, len(paths)-1)
        image = Image.open(TEST/paths[randomNumber])
        
        ax = fig.add_subplot(10, 5, i)
        ax.axis("off")
        
        plt.imshow(image, cmap='gray')
    plt.show()
print('samples of training data')
displayTrainingData()

## Resize images to 48x48

I'll do this because the model crashes with 28x28

In [ ]:
!mkdir train_lrg

In [ ]:
trn_path = Path("/kaggle/working/train_lrg")
resize_images(TRAIN, dest=trn_path, max_size=48, recurse=True)

In [ ]:
!mkdir test_lrg

In [ ]:
test_path = Path("/kaggle/working/test_lrg")
resize_images(TEST, dest=test_path, max_size=224, recurse=True)

In [ ]:
tst_files = get_image_files(test_path).sorted()

## Create data loader

In [ ]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.01,
                                   #item_tfms=Resize(48, method='squish'),
                                   batch_tfms=aug_transforms(max_rotate=20.0, do_flip=False),
                                   bs=1024)

dls.show_batch(max_n=6)

## Define training function

In [ ]:
def train(arch, size, item=Resize(48, method='squish'), accum=1, finetune=True, epochs=12, lr=0.01):
    dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.05,
                                   item_tfms=item,
                                   batch_tfms=aug_transforms(size=size, max_rotate=20.0, do_flip=False),
                                   bs=128//accum)
    cbs = GradientAccumulation(128) if accum else []
    learn = vision_learner(dls, arch, metrics=[error_rate, accuracy], cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, lr)
        return learn.tta(dl=dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, lr)
    
    return learn
        
        
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

## First model

In [ ]:
report_gpu()

In [ ]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.05,
                                   item_tfms=None,
                                   batch_tfms=aug_transforms(size=224, max_rotate=20.0, do_flip=False),
                                   bs=128)
#learn = vision_learner(dls, 'vit_base_patch16_224_miil', metrics=[error_rate, accuracy]).to_fp16()

In [ ]:
learn.lr_find(suggest_funcs=(valley, slide))

In [ ]:
train('convnext_small.fb_in22k', size=None, item=None, epochs=1, accum=1, finetune=False)
report_gpu()

In [ ]:
train('convnext_large_in22k', size=None, item=None, epochs=1, accum=2, finetune=False)
report_gpu()

In [ ]:
train('regnetx_120', size=None, item=None, epochs=1, accum=1, finetune=False)
report_gpu()

In [ ]:
train('vit_base_patch16_224_miil', size=224, item=None, epochs=1, accum=1, finetune=False)
report_gpu()

In [ ]:
train('regnety_320.swag_ft_in1k', size=None, item=None, epochs=1, accum=1, finetune=False)
report_gpu()

In [ ]:
train('beit_large_patch16_224.in22k_ft_in22k_in1k', size=224, item=None, epochs=1, accum=16, finetune=False)
report_gpu()

In [ ]:
learn = train('vit_base_patch16_224_miil', size=224, item=None, epochs=15, lr=0.001, accum=1, finetune=False)

In [ ]:
report_gpu()
learn_beit = train('beit_large_patch16_224.in22k_ft_in22k_in1k', size=224, item=None, epochs=5, accum=16, finetune=False)
report_gpu()

In [ ]:
# save results
save_pickle('model_vit_base_patch16_224_miil.pkl', learn)

In [ ]:
learn = load_pickle(Path('/kaggle/input/model-vit-base-patch16-224-miil-digit-recognizer/model_vit_base_patch16_224_miil.pkl'))

In [ ]:
learn.fine_tune(epochs=1, base_lr=0.001, freeze_epochs=0)

# Ensemble models

In [ ]:
models = {
    'beit_large_patch16_224.in22k_ft_in22k_in1k': {
        (size=224, accum=4)
    }, 'vit_base_patch16_224_miil': {
        (size=224, accum=2),
    }, 'regnetx_120': {
        (size=None, accum=1)
}

In [ ]:
# train and predict
tta_res = []

for arch,details in models.items():
    for size, accum in details:
        print('---',arch)
        print(size)
        print(accum)
        tta_res.append(train(arch, size=size, accum=accum, item=None, epochs=12, finetune=True))
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
# save results
save_pickle('tta_res.pkl', tta_res)

## Test time augmentation

To make the predictions even better, we can try test time augmentation (TTA), which our book defines as:

During inference or validation, creating multiple versions of each image, using data augmentation, and then taking the average or maximum of the predictions for each augmented version of the image.

Before trying that out, we'll first see how to check the predictions and error rate of our model without TTA:

In [ ]:
valid = learn.dls.valid
preds,targs = learn.get_preds(dl=valid)


In [ ]:
error_rate(preds, targs)

In [ ]:
learn.dls.train.show_batch(max_n=6, unique=True)


In [ ]:
tta_preds,_ = learn.tta(dl=valid)

In [ ]:
error_rate(tta_preds, targs)

## Submitting to Kaggle

In [ ]:
ss = pd.read_csv(path/'sample_submission.csv')
ss

In [ ]:
tst_files = get_image_files(test_path).sorted()
tst_dl = dls.test_dl(tst_files)


In [ ]:
# with tta
preds,_ = learn.tta(dl=tst_dl)
idxs = preds.argmax(dim=1)

# without tta
#probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)
#idxs

## Have a look at some predictions

In [ ]:
%matplotlib inline
files = get_image_files(TEST).sorted()
for i in range(10):
    img = PILImage.create(files[i])
    img.to_thumb(28)
    img.show()

In [ ]:
order = []
for file in files:
    order.append(int(file.stem)+1)

In [ ]:
ss['Label'] = list(idxs.numpy())
ss['ImageId'] = order 
ss.sort_values(by="ImageId", inplace=True)
ss

In [ ]:
ss.to_csv('subm.csv', index=False)
!head subm.csv

In [ ]:
# Add your PRIVATE credentials
# Do not use "!export KAGGLE_USERNAME= ..." OR "" around your credential
%env KAGGLE_USERNAME=kahnertk
%env KAGGLE_KEY=SmelVUHeti3NUcQ

# Verify
!export -p | grep KAGGLE_USERNAME
!export -p | grep KAGGLE_KEY

In [ ]:
# Submit
!kaggle competitions submit -c digit-recognizer -f subm.csv -m "vit_base_patch16_224_miil for 20 epochs"

# View results
!kaggle competitions submissions -c digit-recognizer

In [ ]:
! rm -r /kaggle/working/*